# Derivative of traits_i with respect to traits_k

Here I will be testing my solution for
$\frac{ \partial \mathbf{V_{i,t+1}} }{ \partial \mathbf{V_{k,t}} }$
(see below)
by calculating the Jacobian using the `theano` package
and comparing those results to my solution.




## Importing packages and setting options

In [2]:
%env OMP_NUM_THREADS=4
%env THEANO_FLAGS='openmp=True'
import sympy
import theano
theano.config.cxx = ""
import theano.tensor as T
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
pd.options.display.max_columns = 10

env: OMP_NUM_THREADS=4
env: THEANO_FLAGS='openmp=True'


## Equations

__Notes:__

- ${}^\text{T}$ represents transpose.
- Elements in __bold__ are matrices
- Multiplication between matrices is always matrix multiplication, not
  element-wise
  

The equations for (1) traits for species $i$ at time $t+1$ ($\mathbf{V}_{i,t+1}$)
and (2) the partial derivative of species $i$ traits with respect
to species $k$ traits (where $k \ne i$)
are as follows:

\begin{align}
\mathbf{V}_{i,t+1} &= \mathbf{V}_{i,t} + 2 ~ \sigma_i^2
    \left[
        \left(
            N_{k,t} ~ \textrm{e}^{-\mathbf{V}_{k,t}^\textrm{T} \mathbf{D} \mathbf{V}_{k,t}} +
            \mathbf{\Phi}_{i,t}
        \right)
        \left(
            \alpha_0 ~ \textrm{e}^{-\mathbf{V}_{i,t}^\textrm{T}
            \mathbf{V}_{i,t}} ~ \mathbf{V}_{i,t}^\textrm{T}
        \right)
        - f \mathbf{V}_{i,t}^\textrm{T} \mathbf{C}
    \right] \\
    \mathbf{\Phi}_{i,t} &= N_{i,t} + \sum_{j \ne i, j \ne k}^{n}{
        N_{j,t} ~ \textrm{e}^{- \mathbf{V}_{j,t}^\textrm{T} \mathbf{D}
        \mathbf{V}_{j,t} } } \\
    \frac{ \partial\mathbf{V}_{i,t+1} }{ \partial\mathbf{V}_{k,t}} &=
        -4 ~ \alpha_0 ~ \sigma^2 ~ N_{k,t} ~ 
        \mathbf{V}_{i,t}
        \textrm{e}^{
                - \mathbf{V}_{k,t}^{\textrm{T}} \mathbf{D} \mathbf{V}_{k,t}
                - \mathbf{V}_{i,t}^{\textrm{T}} \mathbf{V}_{i,t}
            }
        \mathbf{V}_{k,t}^{\textrm{T}} ~ \mathbf{D}
\end{align}


## Read CSV of simulated datasets

In [3]:
sims = pd.read_csv("simulated_data.csv")
sims.head()

,V1,V2,V3,V4,V5,...,f,a0,eta,r0,d
0,4.945110,2.869199,6.747126,6.142522,5.629532,...,0.068890,0.112113,-0.331150,1.422746,-0.091228
1,0.718846,1.220364,0.815571,0.868633,0.838021,...,0.309021,0.057579,0.094811,1.237047,0.003429
2,3.369285,1.912974,3.131174,0.046303,1.416252,...,0.118318,0.401410,-0.036977,1.746024,0.012160
3,0.373669,0.283873,0.237735,0.053632,0.062281,...,0.497286,0.499730,0.117188,0.669199,0.081612
4,3.562637,1.635016,5.724176,4.953962,1.060083,...,0.042638,0.307171,-0.467453,0.952351,0.051834


## Functions to compare methods

In [4]:
def automatic(i, k, N, V, D, f, a0, C, s2):
    """Automatic differentiation using theano pkg"""
    Vi = V[:,i]
    Ni = N[i]
    Nk = N[k]
    P = [np.exp(-1 * np.dot(np.dot(V[:,j].T, D), V[:,j])) * N[j] 
         for j in range(0, N.size) if j != i and j != k]
    P = np.sum(P) + Ni
    Vk_ = T.dvector('Vk_')
    Vhat = Vi + 2 * s2 * ( T.dot(Nk * T.exp(-1 * T.dot(T.dot(Vk_.T, D), Vk_)) + P, 
                              a0 * T.dot(T.exp(-1 * T.dot(Vi.T, Vi)), Vi.T)) -
                       f * T.dot(Vi.T, C) )
    J, updates = theano.scan(lambda i, Vhat, Vk_ : T.grad(Vhat[i], Vk_), 
                         sequences=T.arange(Vhat.shape[0]), non_sequences=[Vhat, Vk_])
    num_fun = theano.function([Vk_], J, updates=updates)
    out_array = num_fun(V[:,k])
    return out_array

In [13]:
def symbolic(i, k, N, V, D, a0, s2):
    """Symbolic differentiation using math"""
    Vi = V[:,i]
    Vi = Vi.reshape((3, 1))
    Vk = V[:,k]
    Vk = Vk.reshape((3, 1))
    Ni = N[i]
    Nk = N[k]
    ZZZ = np.exp(-1 * Vk.T @ D @ Vk - Vi.T @ Vi)
    ZZZ = Vi @ ZZZ @ Vk.T @ D
    dVhat = -4 * a0 * s2 * Nk * ZZZ
    return dVhat

In [14]:
def compare_methods(sim_i, s2 = 0.01, abs = False):
    """Compare answers from symbolic and automatic methods"""
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    n, q = (N.size, int(V.size / N.size))
    V = V.reshape((q, n), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, sims.loc[sim_i,"d"])
    # r0 = sims.loc[sim_i,"r0"]  # don't need this one now
    diffs = np.empty((math.factorial(n) // math.factorial(n-2), 4))
    j = 0
    for i in range(0, n):
        for k in [x for x in range(0, n) if x != i]:
            num = automatic(i, k, N, V, D, f, a0, eta, s2)
            sym = symbolic(i, k, N, V, D, a0, s2)
            num = num.flatten()
            sym = sym.flatten()
            if abs:
                diff = num - sym
            else:
                diff = (num - sym) / sym
                if np.any(sym == 0):
                    for l in [x for x in range(0, diff.size) if sym[x] == 0]:
                        diff[l] = num[l];
            diffs[j, 0] = i
            diffs[j, 1] = k
            diffs[j, 2] = diff.min()
            diffs[j, 3] = diff.max()
            j += 1
    return diffs

### Example of using `compare_methods`:

In [15]:
diffs = compare_methods(0)
print(diffs[:,2].min())
print(diffs[:,3].max())

-6.397022784452073e-15
6.792467255897916e-15


## Comparing methods

This takes ~5-6 minutes.

In [16]:
n_per_rep = math.factorial(4) // math.factorial(4-2)
diffs = np.empty((int(n_per_rep * 100), 4))

In [17]:
for rep in tqdm(range(100)):
    diffs_r = compare_methods(rep)
    diffs[(rep * n_per_rep):((rep+1) * n_per_rep),:] = diffs_r

100%|██████████| 100/100 [10:24<00:00,  6.25s/it]


## The results
They appear to have extremely similar values, similar enough to make me quite comfortable saying that my symbolic solution works.

In [18]:
print(diffs[:,2].min())
print(diffs[:,3].max())

-2.2373345736945266e-14
2.514702717441046e-14


## Write output to file

To make sure the R version works, too, I'm writing to a CSV file the output from the symbolic version on the 100 datasets.

In [19]:
n = np.sum([x.startswith("N") for x in sims.columns])
q = int(np.sum([x.startswith("V") for x in sims.columns]) / n)
s2 = 0.01
n_perms = math.factorial(n) // math.factorial(n-2)
# Output array
results = np.zeros((100, n_perms * q * q))

for sim_i in range(100):
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    V = V.reshape((q, n), order = 'F')
    a0 = sims.loc[sim_i,"a0"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, sims.loc[sim_i,"d"])

    # Fill output array:
    j = 0
    for i in range(0, n):
        for k in [x for x in range(0, n) if x != i]:
            sym = symbolic(i, k, N, V, D, a0, s2)
            results[sim_i, (j*q*q):((j+1)*q*q)] = sym.flatten()
            j += 1

# Make sure the last row isn't zeros:
results[99, :]

array([-6.10780008e-04, -5.23802407e-04, -1.43344210e-04, -2.05168547e-04,
       -1.75951697e-04, -4.81510903e-05, -1.48977726e-04, -1.27762681e-04,
       -3.49636436e-05, -2.81323554e-03, -1.26286013e-03, -8.72343575e-04,
       -9.45000556e-04, -4.24210311e-04, -2.93030979e-04, -6.86187216e-04,
       -3.08029123e-04, -2.12776712e-04, -1.79030789e-05, -3.48683303e-04,
       -2.41055308e-04, -6.01386528e-06, -1.17127027e-04, -8.09734547e-05,
       -4.36680957e-06, -8.50486998e-05, -5.87967373e-05, -5.41178485e-10,
       -1.81788536e-10, -1.32000948e-10, -4.64112429e-10, -1.55901096e-10,
       -1.13203467e-10, -1.27009401e-10, -4.26640260e-11, -3.09793566e-11,
       -5.99265592e-10, -2.69010046e-10, -1.85823576e-10, -5.13927692e-10,
       -2.30701902e-10, -1.59361530e-10, -1.40641888e-10, -6.31340780e-11,
       -4.36110113e-11, -3.81365123e-12, -7.42752972e-11, -5.13487584e-11,
       -3.27057152e-12, -6.36981875e-11, -4.40364827e-11, -8.95027372e-13,
       -1.74317000e-11, -

In [20]:
np.savetxt('results/dVi_dVk.csv', results, delimiter=',')